# Getting Embeddings

In [1]:
from transformers import BertModel, BertTokenizer
from scipy.spatial.distance import cosine
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
print(tokenizer("feline", return_tensors='pt'))
print(len(tokenizer.vocab))
print(tokenizer("cat", return_tensors='pt',  truncation=True))

NameError: name 'tokenizer' is not defined

In [8]:

def get_embeddings(model, tokens):
    with torch.no_grad():
        output = model(**tokens)
        embedding = output.last_hidden_state[0][1]
        print(tokens)
        return torch.reshape(embedding, (768,))


In [116]:
tokens = {
    "medicine": tokenizer("medicine", return_tensors='pt', truncation=True),
    "cat": tokenizer("cat", return_tensors='pt', truncation=True),
    "kitty": tokenizer('kitty', return_tensors='pt', truncation=True),
    "feline": tokenizer('feline', return_tensors='pt', truncation=True),
    "doctor": tokenizer("doctor", return_tensors='pt', truncation=True),
    "medical": tokenizer("medical", return_tensors='pt', truncation=True),
}
embeddings = {}
for key, val in tokens.items():
    embeddings[key] = get_embeddings(model, val)

print("cat-kitty: " + f"{cosine(embeddings['cat'], embeddings['kitty'])}")
print('cat-doctor: ' + f"{cosine(embeddings['cat'], embeddings['doctor'])}")
print('medicine-cat: ' +
      f"{cosine(embeddings['medicine'], embeddings['cat'])}")
print('medicine-medical: ' +
      f"{cosine(embeddings['medicine'], embeddings['medical'])}")


SyntaxError: invalid syntax (890100369.py, line 9)

In [86]:
cat_emb = get_embeddings(cat_outputs)
hello_emb = get_embeddings(hello_outputs)
hi_emb = get_embeddings(hi_outputs)

print(cosine(hi_emb, hello_emb))
print(cosine(hi_emb, cat_emb))

NameError: name 'cat_outputs' is not defined

In [77]:
cosine([1], [0.1])

0

# Building vocab for the corpus

In [77]:
import pandas as pd
import torchtext.transforms as T
import nltk
from pathlib import Path
import re
import numpy as np 
import pickle
data_dir = Path('data')

In [60]:
df = pd.read_csv(data_dir / 'dataset.csv')
df.head()

,category,title,description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


/home/zhang/miniconda3/envs/honours


In [57]:
try:
    nltk.data.find('tokenizers/punkt.zip')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords.zip')
except:
    nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to /home/zhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/zhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zhang/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [61]:
def vocab_preprocess(row, lemmatize=True):
    from nltk.stem import WordNetLemmatizer
    row = row.lower()
    row = row.split('-')[1::]
    row = ''.join(row)
    lemmatizer = WordNetLemmatizer()
    # tokenize words
    words = re.findall(re.compile('[a-zA-Z]+'), row)
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    if (lemmatize):
        words = [lemmatizer.lemmatize(word) for word in words]

    return words

In [78]:
desc = df['description'].astype(str)
data = []
for i, row in desc.iteritems():
    data.append(vocab_preprocess(row))
vocab = set()
for words in data:
    vocab = vocab.union(set(words))
with open("vocab.pkl", "wb") as f:
    pickle.dump(vocab, f)

45394

array([list(['shortsellers', 'wall', 'street', 'dwindling', 'band', 'ultracynics', 'seeing', 'green']),
       list(['private', 'investment', 'firm', 'carlyle', 'group', 'reputation', 'making', 'welltimed', 'occasionally', 'controversial', 'play', 'defense', 'industry', 'quietly', 'placed', 'bet', 'another', 'part', 'market']),
       list(['soaring', 'crude', 'price', 'plus', 'worry', 'economy', 'outlook', 'earnings', 'expected', 'hang', 'stock', 'market', 'next', 'week', 'depth', 'summer', 'doldrums']),
       ..., list([]), list([]), list([])], dtype=object)